## Imports

In [ ]:
import torch
from torch import nn
import optuna
#import torch.nn.functional as F
#import csv
#import pandas as pd

## Model Class

In [ ]:
from architecture import VAE

## Training Parameters

In [ ]:
# Device init
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

W_DECAY = 1e-5
LEARN_RATE = 1e-4
EPOCH_NUM = 20
BATCH_SIZE = 64

BETA = 0.1 # for the KL divergence term
BETA_INCREASE = 0.1

## Data Creation

In [ ]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import torchvision.transforms.v2 as transforms

def create_data():
    # ---INITIALIZE DATASET ---
    #Convert pilimage dataset to a standart numpy dataset
    dataset = MNIST(
        root='./data',
        download=True,  # Add this to download the dataset if needed
        transform= transforms.ToTensor()
    )

    TRIM_LEN = int(10_000)  # 60,000 - 10,000 = 50,000 SAMPLES
    TRAIN_PORTION = 0.95 # 95% training 5% everything else
    TRAIN_LEN = int((len(dataset) - TRIM_LEN) * TRAIN_PORTION)
    
    # ---SPLIT DATASET---
    train_ds, test_ds, _ = random_split(
        dataset,  # Split the dataset, not the dataloader!
        [TRAIN_LEN, len(dataset) - TRIM_LEN - TRAIN_LEN, TRIM_LEN]
    )
    #print(f"train length: {len(train_ds)} test_length: {len(test_ds)}")
    
    # ---CREATE DATALOADERS from the split datasets---
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
    
    return train_loader, test_loader

## Training

In [ ]:
def train_vae(model, train_loader, optimizer, epoch_num):
    # Method 1: One-liner
    #single_batch = next(batch_idx(train_loader))[0].to(device)
    global BETA
    #loss_func = nn.BCELoss(reduction='sum')
    loss_func = nn.MSELoss(reduction='sum')

    for batch_idx, single_batch in enumerate(train_loader):
        model.train()

        optimizer.zero_grad()

        single_batch = single_batch[0].to(device)
        # ---------Feed Forward---------
        # Extract just the generated images for now
        mean,log_var,img_gen_batch = model.forward(single_batch)

        #---------Back Prop---------
        # Loss is calculated by the batch's mean
        
        flat_sample = torch.flatten(single_batch,start_dim=1)
        img_gen_batch = torch.flatten(img_gen_batch,start_dim=-1)[0]

        kl_div = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())#.mean()

        loss = loss_func(img_gen_batch, flat_sample) + BETA*kl_div

        if batch_idx%100==0:
            print(f"batch num {batch_idx}: {loss.item()} at epoch: {epoch_num+1}")
        
        loss.backward()
        optimizer.step()
        

    if (BETA<1.0):
            #BETA*=1 + 1e-4
            BETA+=BETA_INCREASE
    torch.save(model.state_dict(), "vae_model.pth")
        

## Main

In [ ]:
# Initialize the model
my_vae = VAE(device).to(device)
optim = torch.optim.Adam(params=my_vae.parameters(),
                         lr = LEARN_RATE, weight_decay=W_DECAY)

train_loader,test_loader = create_data()


for i in range(EPOCH_NUM):
    train_vae(model=my_vae, train_loader=train_loader,
            optimizer=optim,epoch_num=i)